In [19]:
%load_ext tensorboard

import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from delta.assets import download_assets
from delta.utilities import cfg
from delta.model import unet_seg
from delta.data import trainGenerator_seg, seg_weights

import pathlib

import datetime


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
def to_str(posixpath):
    return str(posixpath.resolve())  

In [3]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter')
data_dir = root / 'data'
model_dir = root / 'models'

new_config_filename = root / 'config_2D_caulobacter.json'

training_set = data_dir / 'training'
validation_set = data_dir / 'validation'

savefile = model_dir / 'Caulobacter2D'

In [23]:
# Load config:
cfg.load_config(new_config_filename)

# Training parameters:
batch_size = 4
epochs = 600
steps_per_epoch = 300
patience = 50

#Data generator parameters:
data_gen_args = dict(
    rotation = 2,
    rotations_90d = True,
    zoom=.15,
    horizontal_flip=True,
    vertical_flip=True,
    histogram_voodoo=True,
    illumination_voodoo=True,
    gaussian_noise = 0.03,
    gaussian_blur = 1
    )

data_gen_args_val = dict(
    rotation = 2,
    rotations_90d = True,
    horizontal_flip=True,
    vertical_flip=True,
    )

# Generator init:
gen_train = trainGenerator_seg(
    batch_size,
    os.path.join(to_str(training_set),'img'),
    os.path.join(to_str(training_set),'seg'),
    os.path.join(to_str(training_set),'wei'),
    augment_params = data_gen_args,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

gen_validate = trainGenerator_seg(
    batch_size,
    os.path.join(validation_set,'img'),
    os.path.join(validation_set,'seg'),
    os.path.join(validation_set,'wei'),
    augment_params = data_gen_args_val,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

# Define model:
model = unet_seg(input_size = cfg.target_size_seg+(1,), pretrained_weights=cfg.model_file_seg)
model.summary()

Loading configuration from: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/config_2D_caulobacter.json
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 true_input (InputLayer)        [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 Level0_Conv2D_1 (Conv2D)       (None, 512, 512, 64  640         ['true_input[0][0]']             
                                )                                                                 
                                                                                                  
 Level0_Conv2D_2 (Conv2D)       (None, 512, 512, 64  36928       ['Level0_Co

In [24]:
a,b = next(gen_validate)

In [25]:
b.shape

(4, 512, 512, 2)

In [26]:
# Callbacks:
model_checkpoint = ModelCheckpoint(
    to_str(savefile), monitor='loss', verbose=2, save_best_only=True
    )
early_stopping = EarlyStopping(
    monitor='loss', mode='min', verbose=2, patience=patience
    )

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensor_board = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [27]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 191148), started 0:00:52 ago. (Use '!kill 191148' to kill it.)

In [ ]:
# Train:
history = model.fit(
    gen_train,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=gen_validate,
    validation_steps=10,
    callbacks=[model_checkpoint, early_stopping, tensor_board]
    )

Epoch 1/600
300/300 [==============================] - ETA: 0s - loss: 10.3204 - unstack_acc: 0.9919
Epoch 1: loss improved from inf to 10.32043, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D
INFO:tensorflow:Assets written to: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D/assets
300/300 [==============================] - 230s 745ms/step - loss: 10.3204 - unstack_acc: 0.9919 - val_loss: 7.5270 - val_unstack_acc: 0.9939
Epoch 2/600


/scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/python3.9/site-packages/delta/data.py:392: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  I = trans.rescale(I, zoomlevel, mode="edge", multichannel=False, order=order)


300/300 [==============================] - ETA: 0s - loss: 8.5856 - unstack_acc: 0.9924
Epoch 2: loss improved from 10.32043 to 8.58564, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D
INFO:tensorflow:Assets written to: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D/assets
300/300 [==============================] - 220s 736ms/step - loss: 8.5856 - unstack_acc: 0.9924 - val_loss: 5.5472 - val_unstack_acc: 0.9951
Epoch 3/600


/scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/python3.9/site-packages/delta/data.py:392: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  I = trans.rescale(I, zoomlevel, mode="edge", multichannel=False, order=order)


300/300 [==============================] - ETA: 0s - loss: 8.0597 - unstack_acc: 0.9926
Epoch 3: loss improved from 8.58564 to 8.05965, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D
INFO:tensorflow:Assets written to: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D/assets
300/300 [==============================] - 220s 734ms/step - loss: 8.0597 - unstack_acc: 0.9926 - val_loss: 6.2649 - val_unstack_acc: 0.9931
Epoch 4/600


...

300/300 [==============================] - ETA: 0s - loss: 6.4525 - unstack_acc: 0.9932
Epoch 26: loss improved from 6.46839 to 6.45249, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D
INFO:tensorflow:Assets written to: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D/assets
300/300 [==============================] - 220s 736ms/step - loss: 6.4525 - unstack_acc: 0.9932 - val_loss: 6.2436 - val_unstack_acc: 0.9924
Epoch 27/600


/scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/python3.9/site-packages/delta/data.py:392: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  I = trans.rescale(I, zoomlevel, mode="edge", multichannel=False, order=order)


300/300 [==============================] - ETA: 0s - loss: 6.5639 - unstack_acc: 0.9931
Epoch 27: loss did not improve from 6.45249
300/300 [==============================] - 217s 723ms/step - loss: 6.5639 - unstack_acc: 0.9931 - val_loss: 5.1168 - val_unstack_acc: 0.9950
Epoch 28/600
300/300 [==============================] - ETA: 0s - loss: 6.4891 - unstack_acc: 0.9934
Epoch 28: loss did not improve from 6.45249
300/300 [==============================] - 215s 719ms/step - loss: 6.4891 - unstack_acc: 0.9934 - val_loss: 6.4053 - val_unstack_acc: 0.9919
Epoch 29/600
300/300 [==============================] - ETA: 0s - loss: 6.2385 - unstack_acc: 0.9936
Epoch 29: loss improved from 6.45249 to 6.23849, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D
INFO:tensorflow:Assets written to: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/Caulobacter2D/assets
300/300 [==============================] - 227s 758ms/step - loss: 6.23

/scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/python3.9/site-packages/delta/data.py:392: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  I = trans.rescale(I, zoomlevel, mode="edge", multichannel=False, order=order)


300/300 [==============================] - ETA: 0s - loss: 6.3428 - unstack_acc: 0.9935
Epoch 30: loss did not improve from 6.23849
300/300 [==============================] - 217s 725ms/step - loss: 6.3428 - unstack_acc: 0.9935 - val_loss: 5.6056 - val_unstack_acc: 0.9946
Epoch 31/600
300/300 [==============================] - ETA: 0s - loss: 6.4561 - unstack_acc: 0.9932
Epoch 31: loss did not improve from 6.23849
300/300 [==============================] - 217s 724ms/step - loss: 6.4561 - unstack_acc: 0.9932 - val_loss: 7.7563 - val_unstack_acc: 0.9952
Epoch 32/600
279/300 [==========================>...] - ETA: 14s - loss: 6.8339 - unstack_acc: 0.9930